In [ ]:
# Completely remove conflicting packages
!pip uninstall -y numpy opencv-python opencv-python-headless torch torchvision torchaudio ultralytics
!pip install --upgrade pip
!pip cache purge


Found existing installation: numpy 2.2.6
Uninstalling numpy-2.2.6:
  Successfully uninstalled numpy-2.2.6
Found existing installation: opencv-python 4.12.0.88
Uninstalling opencv-python-4.12.0.88:
  Successfully uninstalled opencv-python-4.12.0.88
Found existing installation: opencv-python-headless 4.9.0.80
Uninstalling opencv-python-headless-4.9.0.80:
  Successfully uninstalled opencv-python-headless-4.9.0.80
Found existing installation: torch 2.5.1
Uninstalling torch-2.5.1:
  Successfully uninstalled torch-2.5.1
Found existing installation: torchvision 0.20.1
Uninstalling torchvision-0.20.1:
  Successfully uninstalled torchvision-0.20.1
Found existing installation: torchaudio 2.5.1
Uninstalling torchaudio-2.5.1:
  Successfully uninstalled torchaudio-2.5.1
Found existing installation: ultralytics 8.0.172
Uninstalling ultralytics-8.0.172:
  Successfully uninstalled ultralytics-8.0.172
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 38.3 MB/s eta 0:00:00
  Attempting uninstall: p

In [ ]:
# ✅ Install only the compatible stack (no TensorFlow/JAX etc.)
!pip install numpy==1.26.4 torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1 --quiet
!pip install ultralytics==8.0.172 opencv-python-headless==4.9.0.80 ffmpeg-python --quiet


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dopamine-rl 4.1.2 requires opencv-python>=3.4.8.29, which is not installed.
albumentations 2.0.8 requires opencv-python-headless>=4.9.0.80, which is not installed.
albucore 0.0.24 requires opencv-python-headless>=4.9.0.80, which is not installed.
jaxlib 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
pytensor 2.35.1 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
jax 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
datasets 4.0.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.9.0 which is incompati

In [ ]:
# Prevent any library from upgrading NumPy back to 2.x
!pip install "numpy==1.26.4" --upgrade --no-cache-dir --no-deps


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 38.9 MB/s  0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6


In [ ]:
import numpy, torch, cv2
from ultralytics import YOLO

print("✅ NumPy:", numpy.__version__)
print("✅ Torch:", torch.__version__)
print("✅ OpenCV:", cv2.__version__)

model = YOLO("yolov8n.pt")
print("✅ YOLOv8 model loaded successfully!")


/usr/local/lib/python3.12/dist-packages/ultralytics/utils/checks.py:17: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources as pkg


✅ NumPy: 1.26.4
✅ Torch: 2.5.1+cu124
✅ OpenCV: 4.12.0
✅ YOLOv8 model loaded successfully!


/usr/local/lib/python3.12/dist-packages/ultralytics/nn/tasks.py:527: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(file, map_location='cpu'), file  # load


In [ ]:
from google.colab import files

uploaded = files.upload()
input_video_path = list(uploaded.keys())[0]
print(f"🎥 Video uploaded successfully: {input_video_path}")


Saving Elephant.mp4 to Elephant.mp4
🎥 Video uploaded successfully: Elephant.mp4


In [ ]:
import cv2
from ultralytics import YOLO
from pathlib import Path
from IPython.display import HTML, display

MODEL_NAME = "yolov8n.pt"
FRAME_SKIP = 5
CONFIDENCE = 0.25
INPUT = input_video_path
OUTPUT = "elephant_output.mp4"

print("🔄 Loading YOLOv8 model...")
model = YOLO(MODEL_NAME)
print("✅ Model loaded successfully!")

cap = cv2.VideoCapture(INPUT)
if not cap.isOpened():
    raise RuntimeError(f"❌ Could not open video: {INPUT}")

fps = cap.get(cv2.CAP_PROP_FPS) or 25.0
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(OUTPUT, fourcc, fps, (width, height))

frame_idx, elephant_found, first_time = 0, False, None

print("🚀 Detecting elephants... please wait...")

while True:
    ret, frame = cap.read()
    if not ret:
        break
    if frame_idx % FRAME_SKIP == 0:
        results = model.predict(source=frame, conf=CONFIDENCE, verbose=False)
        for box in results[0].boxes:
            cls_id = int(box.cls.cpu().numpy())
            cls_name = model.names.get(cls_id, str(cls_id))
            if cls_name.lower() == "elephant":
                elephant_found = True
                timestamp = frame_idx / fps
                first_time = f"{int(timestamp//60)}m {int(timestamp%60)}s"
                xyxy = box.xyxy[0].cpu().numpy().astype(int)
                cv2.rectangle(frame, (xyxy[0], xyxy[1]), (xyxy[2], xyxy[3]), (0,255,0), 2)
                label = f"Elephant {float(box.conf.cpu()):.2f}"
                cv2.putText(frame, label, (xyxy[0], xyxy[1]-10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)
                break
        if elephant_found:
            print(f"🚨 Elephant detected at {first_time}!")
            break
    out.write(frame)
    frame_idx += 1

cap.release()
out.release()

if elephant_found:
    print("\n🚨🚨🚨 EMERGENCY ALERT: ELEPHANT DETECTED! 🚨🚨🚨")
    print(f"📸 Detected at {first_time} into the video.")
    print(f"📁 Output saved as: {OUTPUT}")
    display(HTML(f"<h3 style='color:red;'>🚨 Elephant detected at {first_time}! 🚨</h3>"))
else:
    print("\n✅ No elephant detected in the video.")
    display(HTML("<h3 style='color:green;'>✅ No Elephant Detected ✅</h3>"))


🔄 Loading YOLOv8 model...
✅ Model loaded successfully!
🚀 Detecting elephants... please wait...


/usr/local/lib/python3.12/dist-packages/ultralytics/nn/tasks.py:527: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(file, map_location='cpu'), file  # load


🚨 Elephant detected at 0m 41s!

🚨🚨🚨 EMERGENCY ALERT: ELEPHANT DETECTED! 🚨🚨🚨
📸 Detected at 0m 41s into the video.
📁 Output saved as: elephant_output.mp4


In [ ]:
from google.colab import files
files.download("elephant_output.mp4")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install twilio --quiet


In [ ]:
# 🔐 Replace the below placeholders with your actual Twilio details
TWILIO_ACCOUNT_SID = "YOUR_ACCOUNT_SID_HERE"
TWILIO_AUTH_TOKEN  = "YOUR_AUTH_TOKEN_HERE"
WHATSAPP_FROM      = "whatsapp:+14155238886"   # Twilio sandbox sender number
WHATSAPP_TO        = "whatsapp:+91XXXXXXXXXX"  # Your verified WhatsApp number (include +countrycode)

# Test message
from twilio.rest import Client
twilio_client = Client(TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN)
message = twilio_client.messages.create(
    body="✅ Twilio WhatsApp test successful!",
    from_=WHATSAPP_FROM,
    to=WHATSAPP_TO
)
print("✅ WhatsApp test message sent! SID:", message.sid)


✅ WhatsApp test message sent! SID: SMc70bff0ba8463dd285d795bd4db18384


In [ ]:
import cv2
from ultralytics import YOLO
from twilio.rest import Client
from IPython.display import HTML, display

# --- Your video uploaded previously ---
INPUT = input_video_path
OUTPUT = "elephant_output.mp4"

# --- Model setup ---
model = YOLO("yolov8n.pt")
CONFIDENCE = 0.25
FRAME_SKIP = 5

# --- Twilio setup ---
client = Client(TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN)

def send_whatsapp_alert(text):
    try:
        msg = client.messages.create(
            body=text,
            from_=WHATSAPP_FROM,
            to=WHATSAPP_TO
        )
        print("✅ WhatsApp alert sent! SID:", msg.sid)
    except Exception as e:
        print("⚠️ WhatsApp alert failed:", e)

# --- Detection process ---
cap = cv2.VideoCapture(INPUT)
if not cap.isOpened():
    raise RuntimeError(f"Could not open video: {INPUT}")

fps = cap.get(cv2.CAP_PROP_FPS) or 25.0
width, height = int(cap.get(3)), int(cap.get(4))
out = cv2.VideoWriter(OUTPUT, cv2.VideoWriter_fourcc(*"mp4v"), fps, (width, height))

elephant_found = False
frame_idx = 0
det_time = None

print("🚀 Scanning video for elephants...")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    if frame_idx % FRAME_SKIP == 0:
        results = model.predict(source=frame, conf=CONFIDENCE, verbose=False)
        for box in results[0].boxes:
            cls_id = int(box.cls.cpu().numpy())
            cls_name = model.names.get(cls_id, str(cls_id))
            if cls_name.lower() == "elephant":
                elephant_found = True
                seconds = frame_idx / fps
                det_time = f"{int(seconds//60)}m {int(seconds%60)}s"
                xyxy = box.xyxy[0].cpu().numpy().astype(int)
                cv2.rectangle(frame, (xyxy[0], xyxy[1]), (xyxy[2], xyxy[3]), (0,255,0), 2)
                label = f"Elephant {float(box.conf.cpu()):.2f}"
                cv2.putText(frame, label, (xyxy[0], xyxy[1]-10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)
                break
        if elephant_found:
            print(f"🚨 Elephant detected at {det_time}!")
            break
    out.write(frame)
    frame_idx += 1

cap.release()
out.release()

# --- Final result ---
if elephant_found:
    alert_msg = f"🚨 ALERT: Elephant detected at {det_time} in video '{INPUT}'!"
    print("\n🚨 EMERGENCY ALERT: ELEPHANT DETECTED!")
    display(HTML(f"<h3 style='color:red;'>{alert_msg}</h3>"))
    send_whatsapp_alert(alert_msg)
else:
    print("\n✅ No elephants detected.")
    send_whatsapp_alert(f"No elephants found in video '{INPUT}'.")


🚀 Scanning video for elephants...


/tmp/ipython-input-8856270.py:52: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  cls_id = int(box.cls.cpu().numpy())


🚨 Elephant detected at 0m 41s!

🚨 EMERGENCY ALERT: ELEPHANT DETECTED!


✅ WhatsApp alert sent! SID: SMdd8cddd077610ebb31a9805bb7c97eb9
